# Verify Legal Properties

This notebook is used to verify whether a specification satisfies a property. The notebook evaluates two separate prompt templates: a generic template that refers to a scenario that satisfies a statement; and a requirements template that refers to a specification that satisfies a requirement. The notebook user can switch between the different model variants before obtaining their results.

In [1]:
import json

# load the design properties
properties = json.load(open('results/properties.json', 'r'))

# load the ground truth data
dataset = json.load(open('results/testing.json', 'r'))

In [2]:
from langchain.chat_models import ChatOpenAI
import os

model_names = {
    'gpt35': 'gpt-3.5-turbo-1106',
    'gpt4': 'gpt-4-0613',
    'gpt4p': 'gpt-4-1106-preview'
}

model_name = 'gpt35'

model = ChatOpenAI(
    openai_api_key = os.environ["OPENAI_API_KEY"],
    model_name=model_names[model_name]
)

## Template 1: Scenario and Statement

In [3]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

messages = [
    ('system', 'You are a helpful assistant.'),
    ('human', """Definition of {property}: {definition}
    
Read the following scenario, and decide if the given statement is true or false based on the definition, above. Respond with True or False. Do not comment or elaborate.

Scenario: {base-spec} {prop-actions}

Statement: {requirement}

Answer: """)
]

prompt1 = ChatPromptTemplate.from_messages(messages)
chain = prompt1 | model

inverse = {'True': 'False', 'False': 'True'}
prop_state_str = {'T': 'True', 'F': 'False'}
answers = []

for i in range(0, 10):
    for j, record in enumerate(dataset):
        response = chain.invoke({
            'property': properties[record['prop-code']]['property'],
            'definition': properties[record['prop-code']]['rubric'],
            'base-spec': record['base-spec'],
            'prop-actions': record['prop-actions'],
            'requirement': properties[record['prop-code']]['axiom']['T'] # positive
        })
        answers.append([
            record['id'],
            i,
            record['prop-code'], # property code
            record['prop-state'], # spec polarity
            prop_state_str[record['prop-state']], # expected
            response.content # predicted
        ])

        response = chain.invoke({
            'property': properties[record['prop-code']]['property'],
            'definition': properties[record['prop-code']]['rubric'],
            'base-spec': record['base-spec'],
            'prop-actions': record['prop-actions'],
            'requirement': properties[record['prop-code']]['axiom']['F'] # positive
        })
        answers.append([
            record['id'],
            i,
            record['prop-code'], # property code
            record['prop-state'], # spec polarity
            inverse[prop_state_str[record['prop-state']]], # expected
            response.content # predicted
        ])
        print(j)
    print('### %i ###' % i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
### 7 ###
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91


In [4]:
json.dump(answers, open('results/answers_%s_tempG.json' % model_name, 'w+'))

## Template 2: Specification and Requirement

In [5]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

messages = [
    ('system', 'You are a helpful assistant.'),
    ('human', """Definition of {property}: {definition}
    
Read the following specification. If the specification satisifies the requirement based on the definition, above, respond with True, otherwise respond with False. Do not comment or elaborate.

Specification: {base-spec} {prop-actions}

Requirement: {requirement}

Answer: """)
]

prompt1 = ChatPromptTemplate.from_messages(messages)
chain = prompt1 | model

inverse = {'True': 'False', 'False': 'True'}
prop_state_str = {'T': 'True', 'F': 'False'}
answers = []

for i in range(0, 5):
    for j, record in enumerate(dataset):
        response = chain.invoke({
            'property': properties[record['prop-code']]['property'],
            'definition': properties[record['prop-code']]['rubric'],
            'base-spec': record['base-spec'],
            'prop-actions': record['prop-actions'],
            'requirement': properties[record['prop-code']]['axiom']['T'] # positive
        })
        answers.append([
            record['id'],
            i,
            record['prop-code'], # property code
            record['prop-state'], # spec polarity
            prop_state_str[record['prop-state']], # expected
            response.content # predicted
        ])

        response = chain.invoke({
            'property': properties[record['prop-code']]['property'],
            'definition': properties[record['prop-code']]['rubric'],
            'base-spec': record['base-spec'],
            'prop-actions': record['prop-actions'],
            'requirement': properties[record['prop-code']]['axiom']['F'] # positive
        })
        answers.append([
            record['id'],
            i,
            record['prop-code'], # property code
            record['prop-state'], # spec polarity
            inverse[prop_state_str[record['prop-state']]], # expected
            response.content # predicted
        ])
        print(j)
    print('### %i ###' % i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27